In [11]:
import operator
import random
import pickle
import math
import time
from functools import partial
from collections import Counter

import numpy as np
import pandas as pd
import tqdm

from deap import base, creator, tools, gp

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
EPS = 1e-12


# ========== 工具 ==========
def t_as(x, ref=None):
    """转 torch.float32 张量并放到 ref 的 device/shape 可广播。"""
    if torch.is_tensor(x):
        return x
    return torch.tensor(x, dtype=torch.float32, device=(ref.device if (ref is not None and torch.is_tensor(ref)) else device))

def t_win_from(x, default=10, lo=2, hi=60):
    """把窗口参数变成标量窗口长度（仿 _win_from，对张量取 abs 的 nanmedian）。"""
    x = t_as(x)
    try:
        n = torch.nanmedian(torch.abs(x)).item()
    except Exception:
        n = float(x) if not torch.is_tensor(x) else float(x.item())
    if not torch.isfinite(n):
        n = default
    n = int(max(lo, min(hi, int(abs(n)))))
    return n

def _windows_1d(x, win):
    """
    生成滚动窗口：返回 windows[T, win]，每行对应 x[t-win+1:t+1]，
    左侧用 NaN 填充，等价于 pandas rolling(min_periods=1) 的对齐。
    """
    x = t_as(x)
    T = x.shape[0]
    if win <= 0:
        raise ValueError("win must be positive")
    pad = torch.full((win-1,), float('nan'), dtype=x.dtype, device=x.device)
    xpad = torch.cat([pad, x], dim=0)                   # [T+win-1]
    w = xpad.unfold(0, win, 1)                          # [T, win]
    return w

def _nanmean(w, dim):
    s = torch.nansum(w, dim=dim)
    c = torch.sum(torch.isfinite(w), dim=dim)
    return s / torch.clamp(c, min=1)

def _nanvar(w, dim, ddof=0):
    m = _nanmean(w, dim=dim, )
    diff = w - m.unsqueeze(dim)
    diff = torch.where(torch.isfinite(w), diff, torch.tensor(float('nan'), device=w.device))
    num = torch.nansum(diff * diff, dim=dim)
    c = torch.sum(torch.isfinite(w), dim=dim)
    denom = c - ddof
    denom = torch.clamp(denom, min=1)  # 避免除 0；当 c<=ddof 时结果会不准，但我们随后会在上层置 NaN
    out = num / denom
    out = torch.where(c > ddof, out, torch.tensor(float('nan'), device=w.device))
    return out

def _nanprod(w, dim):
    # 有 NaN 则结果 NaN；否则做正常乘积
    has_nan = ~torch.isfinite(w)
    any_nan = has_nan.any(dim=dim)
    # 把 NaN 替换成 1 再做 prod
    w2 = torch.where(torch.isfinite(w), w, torch.ones_like(w))
    p = torch.prod(w2, dim=dim)
    p = torch.where(any_nan, torch.tensor(float('nan'), device=w.device), p)
    return p

# ========== 基础算术 ==========
def safe_truediv(a, b):
    a = t_as(a, b); b = t_as(b, a)
    return a / (b.abs() + EPS)

def gp_add(a,b):  return t_as(a, b) + t_as(b, a)
def gp_sub(a,b):  return t_as(a, b) - t_as(b, a)
def gp_mul(a,b):  return t_as(a, b) * t_as(b, a)

def gp_sin(x):    return torch.sin(t_as(x))
def gp_cos(x):    return torch.cos(t_as(x))
def gp_tan(x):    return torch.tan(t_as(x)) 
def gp_vneg(x):   x = t_as(x); return -x
def gp_abs(x):    x = t_as(x); return torch.abs(x)
def gp_log(x):    x = t_as(x); return torch.log(torch.abs(x) + EPS)
def gp_sign(x):   x = t_as(x); return torch.sign(x)

def gp_max(a, b): a = t_as(a, b); b = t_as(b, a); return torch.maximum(a, b)
def gp_min(a, b): a = t_as(a, b); b = t_as(b, a); return torch.minimum(a, b)

# ========== 时序延迟/差分 ==========
def _delay(x, k: int):
    x = t_as(x)
    T = x.shape[0]
    y = torch.empty_like(x)
    if k <= 0:
        return x
    y[:k] = float('nan')
    y[k:] = x[:-k]
    return y

def gp_delay_1(x): return _delay(x, 1)
def gp_delay_2(x): return _delay(x, 2)
def gp_delay_3(x): return _delay(x, 3)
def gp_delay_4(x): return _delay(x, 4)
def gp_delay_5(x): return _delay(x, 5)
def gp_delay_6(x): return _delay(x, 6)
def gp_delay_7(x): return _delay(x, 7)
def gp_delay_8(x): return _delay(x, 8)

def gp_delta(x):
    x = t_as(x)
    y = torch.full_like(x, float('nan'))
    y[1:] = x[1:] - x[:-1]
    return y

def gp_signedpower(x):
    x = t_as(x)
    return torch.sign(x) * torch.pow(torch.abs(x) + EPS, 0.5)  # 和你一致

# ========== 平滑/衰减（EMA） ==========
def _ema(x, span: int):
    """纯 torch EMA：alpha=2/(span+1)，min_periods=1 行为。"""
    x = t_as(x)
    span = int(span)
    alpha = 2.0 / (span + 1.0)
    y = torch.empty_like(x)
    # 处理 NaN：沿用 pandas 行为，NaN 参与会传播；这里简单用前值策略避 NaN 爆炸
    prev = torch.nan_to_num(x[0], nan=0.0)
    y[0] = prev
    for t in range(1, x.shape[0]):
        xt = torch.nan_to_num(x[t], nan=prev)  # 用前值填 NaN
        prev = alpha * xt + (1 - alpha) * prev
        y[t] = prev
    return y

def gp_decay_05(x): return _ema(x, 5)
def gp_decay_10(x): return _ema(x, 10)
def gp_decay_20(x): return _ema(x, 20)

# ========== 逻辑/比较/条件 ==========
def gp_and(a, b): return ((t_as(a) > 0) & (t_as(b) > 0)).float()
def gp_or(a, b):  return ((t_as(a) > 0) | (t_as(b) > 0)).float()
def gp_lt(a, b):  return (t_as(a) < t_as(b)).float()
def gp_gt(a, b):  return (t_as(a) > t_as(b)).float()

def gp_if(cond, x, y):
    c = t_as(cond); x = t_as(x, c); y = t_as(y, c)
    return torch.where(c > 0, x, y)

def gp_if_then_else(a, b, x, y):
    aa = t_as(a); bb = t_as(b)
    x = t_as(x, aa); y = t_as(y, aa)
    return torch.where(aa > bb, x, y)

def gp_clear_by_cond(x, y, cond):
    # cond>0 取 y，否则取 x（和你保持一致）
    c = t_as(cond)
    x = t_as(x, c); y = t_as(y, c)
    return torch.where(c > 0, y, x)

# ========== 滚动统计 ==========
def gp_stddev_05(x):
    w = _windows_1d(x, 5)
    # pandas rolling.std 默认 ddof=1；窗口长度=1 时给 NaN
    c = torch.sum(torch.isfinite(w), dim=1)
    v = _nanvar(w, dim=1, ddof=1)
    v = torch.where(c > 1, v, torch.tensor(float('nan'), device=w.device))
    return torch.sqrt(v)

def gp_stddev_10(x):
    w = _windows_1d(x, 10)
    c = torch.sum(torch.isfinite(w), dim=1)
    v = _nanvar(w, dim=1, ddof=1)
    v = torch.where(c > 1, v, torch.tensor(float('nan'), device=w.device))
    return torch.sqrt(v)

def gp_stddev_15(x):
    w = _windows_1d(x, 15)
    c = torch.sum(torch.isfinite(w), dim=1)
    v = _nanvar(w, dim=1, ddof=1)
    v = torch.where(c > 1, v, torch.tensor(float('nan'), device=w.device))
    return torch.sqrt(v)

def gp_prod_05(x):
    w = _windows_1d(x, 5)
    return _nanprod(w, dim=1)

def gp_prod_10(x):
    w = _windows_1d(x, 10)
    return _nanprod(w, dim=1)

def gp_prod_20(x):
    w = _windows_1d(x, 20)
    return _nanprod(w, dim=1)

def gp_wma(x, w, n):
    x = t_as(x); w = t_as(w, x)
    win = t_win_from(n, default=10)
    Xw = _windows_1d(x, win)                 # [T, win]
    Ww = _windows_1d(torch.abs(w) + EPS, win)
    valid = torch.isfinite(Xw) & torch.isfinite(Ww)
    Xw2 = torch.where(valid, Xw, torch.tensor(0.0, device=x.device))
    Ww2 = torch.where(valid, Ww, torch.tensor(0.0, device=x.device))
    num = torch.sum(Xw2 * Ww2, dim=1)
    den = torch.sum(Ww2, dim=1)
    out = num / torch.clamp(den, min=EPS)
    # 若全无有效权重，设为 NaN
    out = torch.where(den > 0, out, torch.tensor(float('nan'), device=x.device))
    return out

def _rolling_cov(a, b, win):
    A = _windows_1d(a, win)  # [T, win]
    B = _windows_1d(b, win)
    val = torch.isfinite(A) & torch.isfinite(B)
    # 均值（对有效项）
    cnt = torch.sum(val, dim=1)
    A2 = torch.where(val, A, torch.tensor(float('nan'), device=A.device))
    B2 = torch.where(val, B, torch.tensor(float('nan'), device=B.device))
    Am = _nanmean(A2, dim=1)
    Bm = _nanmean(B2, dim=1)
    dA = torch.where(val, A - Am.unsqueeze(1), torch.tensor(0.0, device=A.device))
    dB = torch.where(val, B - Bm.unsqueeze(1), torch.tensor(0.0, device=B.device))
    num = torch.sum(dA * dB, dim=1)
    cov = num / torch.clamp(cnt - 1, min=1)  # ddof=1
    cov = torch.where(cnt > 1, cov, torch.tensor(float('nan'), device=A.device))
    return cov

def gp_cov_05(a, b): return _rolling_cov(t_as(a), t_as(b), 5)
def gp_cov_10(a, b): return _rolling_cov(t_as(a), t_as(b), 10)
def gp_cov_20(a, b): return _rolling_cov(t_as(a), t_as(b), 20)

def _rolling_corr(a, b, win):
    cov = _rolling_cov(a, b, win)
    std_a = torch.sqrt(_nanvar(_windows_1d(a, win), dim=1, ddof=1))
    std_b = torch.sqrt(_nanvar(_windows_1d(b, win), dim=1, ddof=1))
    out = cov / (std_a * std_b + EPS)
    # 如果样本数<=1 或 std=0，则前面的 cov/std 也会是 NaN/无穷；保持 NaN 即可
    return out

def gp_corr_05(a, b): return _rolling_corr(t_as(a), t_as(b), 5)
def gp_corr_10(a, b): return _rolling_corr(t_as(a), t_as(b), 10)
def gp_corr_20(a, b): return _rolling_corr(t_as(a), t_as(b), 20)

# ========== 多输入均值 ==========
def gp_mean2(a,b):                        return torch.nanmean(torch.stack([t_as(a), t_as(b)]), dim=0)
def gp_mean3(a,b,c):                     return torch.nanmean(torch.stack([t_as(a), t_as(b), t_as(c)]), dim=0)
def gp_mean4(a,b,c,d):                   return torch.nanmean(torch.stack([t_as(a), t_as(b), t_as(c), t_as(d)]), dim=0)
def gp_mean5(a,b,c,d,e):                 return torch.nanmean(torch.stack([t_as(a), t_as(b), t_as(c), t_as(d), t_as(e)]), dim=0)
def gp_mean6(a,b,c,d,e,f):               return torch.nanmean(torch.stack([t_as(a), t_as(b), t_as(c), t_as(d), t_as(e), t_as(f)]), dim=0)
def gp_mean7(a,b,c,d,e,f,g):             return torch.nanmean(torch.stack([t_as(a), t_as(b), t_as(c), t_as(d), t_as(e), t_as(f), t_as(g)]), dim=0)

# ========== 百分位 rank（滚动窗口） ==========
def rank_x1(x, w1, w2=None, w3=None):
    """
    窗口来自 w1：n=t_win_from(w1)。
    返回每时刻 t：x[t] 在窗口 x[t-n+1:t+1] 内的百分位（<= 的比例）。
    """
    x = t_as(x)
    n = t_win_from(w1, default=10)
    W = _windows_1d(x, n)                     # [T, n]
    last = W[:, -1].unsqueeze(1)              # [T, 1]
    valid = torch.isfinite(W)
    # 计数 <= 最后一个值（含 ties），仅在有效元素上统计
    le = (W <= last) & valid
    cnt = valid.sum(dim=1).clamp(min=1)
    rank = le.sum(dim=1).float() / cnt.float()
    return rank

In [20]:
# =========================
# 配置全局参数
# =========================
# # """
N_GENERATIONS = 6
POP_SIZE = 2000
MUT_PB, MUT_Point, CXPB = 0.05, 0.4, 0.5  # 子树变异率/点变异率/交叉率
NUM_HOF = 200  # 名人堂：精英个体进入名人堂的数目
hall_of_fame = tools.HallOfFame(NUM_HOF)
NUM_ELITE = 200  # 精英策略：精英个体跳过自然选择的数目
tournament_size = 5  # 锦标赛压力
# # """

# 试运行版

# N_GENERATIONS = 5
# POP_SIZE = 10
# MUT_PB, MUT_Point, CXPB = 0.05, 0.4, 0.5
# NUM_HOF = 1
# hall_of_fame = tools.HallOfFame(NUM_HOF)
# NUM_ELITE = 1
# tournament_size = 5


# 配置文件提取储存路径
Ver = 'V7'
path01 = 'Trees/Factors_selected.csv'  # 选取因子代码提取路径
path02 = 'gp/V7/'                     # 种群储存路径
path03 = 'd:/JLProject/factors/alpha191/alphas191_data.csv'  # 因子数据提取路径
path04 = 'gp/V7/HOF.csv'              # 名人堂结果存储路径

# =========================
# 定义因子
# =========================
# factors = pd.read_csv(path01)
# new_factors = factors['Alpha_Index'].iloc[20:28].to_list()  # 新引入因子

# 加入优秀因子（前代高重要性因子/量价数据）
exellent = [
    'alpha052', 'alpha013', 'alpha002', 'alpha191',
    'alpha135', 'alpha022', 'alpha096', 'alpha162', 'alpha038',
    'alpha164', 'alpha049', 'alpha146', 'alpha066', 'alpha161', 'alpha183'
]
base_data = ['open_interest','amount','volume','open']
factors_selected = exellent + base_data
FACTOR_COUNT = len(factors_selected)

print(f"共计{FACTOR_COUNT}个因子")
print(f"保留{len(exellent)}个优秀因子：{exellent}")
#print(f"新引入{len(new_factors)}个新因子：{new_factors}")

# =========================
# 创建 DEAP 的基本结构
# =========================
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0, -1.0, -1.0))  # 多目标：IC均值绝对值最大化，IC标准差最小化，空值最小化, 复杂度最小化
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMulti)   # 定义个体是符号树

# 注册遗传算法工具
toolbox = base.Toolbox()

# 定义 Primitive Set（用于定义操作符 primitive 和变量 terminal）
pset = gp.PrimitiveSet("MAIN", FACTOR_COUNT)
for factor in factors_selected:
    pset.renameArguments(**{f'ARG{factors_selected.index(factor)}': factor})

# —— 基础算子与常量（自定义原语函数请确保在环境中可用） ——
pset.addPrimitive(gp_add, 2)                    # operator.add -> gp_add
pset.addPrimitive(gp_sub, 2)                    # operator.sub -> gp_sub
pset.addPrimitive(gp_mul, 2)                    # operator.mul -> gp_mul
pset.addPrimitive(safe_truediv, 2)              # 你的安全除法（torch 版）

pset.addPrimitive(gp_sin, 1)                    # np.sin -> torch 版
pset.addPrimitive(gp_cos, 1)                    # np.cos -> torch 版
pset.addPrimitive(gp_tan, 1)                    # np.tan -> torch 版

# ==== 用“GPU友好”版本重新注册 ====
pset.addPrimitive(gp_add, 2)                    # operator.add -> gp_add
pset.addPrimitive(gp_sub, 2)                    # operator.sub -> gp_sub
pset.addPrimitive(gp_mul, 2)                    # operator.mul -> gp_mul
pset.addPrimitive(safe_truediv, 2)              # 你的安全除法（torch 版）

pset.addPrimitive(gp_sin, 1)                    # np.sin -> torch 版
pset.addPrimitive(gp_cos, 1)                    # np.cos -> torch 版
pset.addPrimitive(gp_tan, 1)                    # np.tan -> torch 版

# Ephemeral 常量：返回 Python 标量即可；下游算子里会用 t_as 放到正确 device
pset.addEphemeralConstant("rand",     lambda: float(np.random.uniform(0, 20.0)))
pset.addEphemeralConstant("rand_int", lambda: int(random.randint(1, 10)))

# 你已有的“时序/逻辑/滚动/聚合”等 GPU 版函数原样注册
pset.addPrimitive(gp_delay_1, 1)
pset.addPrimitive(gp_delay_2, 1)
pset.addPrimitive(gp_delay_3, 1)
pset.addPrimitive(gp_delay_4, 1)
pset.addPrimitive(gp_delay_5, 1)
pset.addPrimitive(gp_delay_6, 1)
pset.addPrimitive(gp_delay_7, 1)
pset.addPrimitive(gp_delay_8, 1)
pset.addPrimitive(gp_delta, 1)
pset.addPrimitive(gp_signedpower, 1)

pset.addPrimitive(gp_decay_05, 1)
pset.addPrimitive(gp_decay_10, 1)
pset.addPrimitive(gp_decay_20, 1)

pset.addPrimitive(gp_and, 2)
pset.addPrimitive(gp_or,  2)
pset.addPrimitive(gp_lt,  2)
pset.addPrimitive(gp_gt,  2)
pset.addPrimitive(gp_if,  3)

pset.addPrimitive(gp_max, 2)
pset.addPrimitive(gp_min, 2)

pset.addPrimitive(gp_stddev_05, 1)
pset.addPrimitive(gp_stddev_10, 1)
pset.addPrimitive(gp_stddev_15, 1)

pset.addPrimitive(gp_log,  1)
pset.addPrimitive(gp_abs,  1)

pset.addPrimitive(gp_prod_05, 1)
pset.addPrimitive(gp_prod_10, 1)
pset.addPrimitive(gp_prod_20, 1)

pset.addPrimitive(gp_wma, 3)

pset.addPrimitive(gp_vneg, 1)
pset.addPrimitive(gp_sign, 1)

pset.addPrimitive(gp_cov_05,  2)
pset.addPrimitive(gp_cov_10,  2)
pset.addPrimitive(gp_cov_20,  2)
pset.addPrimitive(gp_corr_05, 2)
pset.addPrimitive(gp_corr_10, 2)
pset.addPrimitive(gp_corr_20, 2)

pset.addPrimitive(gp_mean2, 2)
pset.addPrimitive(gp_mean3, 3)
pset.addPrimitive(gp_mean4, 4)
pset.addPrimitive(gp_mean5, 5)
pset.addPrimitive(gp_mean6, 6)
pset.addPrimitive(gp_mean7, 7)

pset.addPrimitive(gp_clear_by_cond, 3)
pset.addPrimitive(gp_if_then_else,  4)

pset.addPrimitive(rank_x1, 4)

# =========================
# 适应度评估（总适应度，多合约合并）
# =========================
def evaluate_individual(individual, func, gpu_cache):
    """
    评估全局个体适应度（多个合约的数据），GPU 版（稳定相关系数）。
    """
    try:
        # 1) 编译个体
        compiled_func = func(individual)

        # 2) 至少包含一个选定因子，否则惩罚
        tree_str = str(individual)
        has_factor = False
        for factor in factors_selected:
            if factor in tree_str:
                has_factor = True
                break
        if not has_factor:
            return (float("-inf"), float("inf"), float("inf"), float("inf"))

        # 3) 逐合约在 GPU 上计算预测，堆成 [T, N]
        names = gpu_cache['_names']
        preds, rets = [], []
        with torch.no_grad():
            for name in names:
                Xt, Yt = gpu_cache[name]     # Xt: list of [T] Tensors; Yt: [T]
                p = compiled_func(*Xt)       # 期望 [T]

                # —— 强制输出与 Yt 对齐 dtype/device，避免 CPU↔GPU 往返
                if not torch.is_tensor(p):
                    p = torch.as_tensor(p, dtype=Yt.dtype, device=Yt.device)
                else:
                    p = p.to(device=Yt.device, dtype=Yt.dtype, non_blocking=True)

                if p.dim() == 0:
                    p = p.expand_as(Yt)      # 防个别原语意外返回标量
                if p.shape[0] != Yt.shape[0]:
                    return (float("-inf"), float("inf"), float("inf"), float("inf"))

                preds.append(p)
                rets.append(Yt)

            P = torch.stack(preds, dim=1).float()  # [T, N]
            R = torch.stack(rets,  dim=1).float()  # [T, N]

        # 4) 逐时点截面 Pearson IC —— 稳定版（严格落在 [-1,1]）
        M = (~torch.isnan(P)) & (~torch.isnan(R))   # 有效掩码 [T, N]
        n = M.sum(dim=1)                            # 每时点有效合约数 [T]

        # 用掩码做和；无效位置置 0 不影响加总
        zeroP = torch.zeros((), device=P.device, dtype=P.dtype)
        zeroR = torch.zeros((), device=R.device, dtype=R.dtype)
        Pz = torch.where(M, P, zeroP)
        Rz = torch.where(M, R, zeroR)

        sumP  = Pz.sum(dim=1)
        sumR  = Rz.sum(dim=1)
        sumP2 = (Pz * Pz).sum(dim=1)
        sumR2 = (Rz * Rz).sum(dim=1)
        sumPR = (Pz * Rz).sum(dim=1)

        num  = n * sumPR - sumP * sumR
        varP = n * sumP2 - sumP * sumP
        varR = n * sumR2 - sumR * sumR

        den = torch.sqrt(torch.clamp(varP, min=0.0) * torch.clamp(varR, min=0.0))

        # 仅在“有效”时点计算；其余置 NaN（不加 epsilon，不硬除）
        valid = (n >= 2) & (varP > 0) & (varR > 0) & torch.isfinite(den)
        ic = torch.full((P.shape[0],), float('nan'), device=P.device)
        ic[valid] = (num[valid] / den[valid]).clamp(-1.0, 1.0)

        # 5) 质量门槛与统计（NaN 友好）
        T = ic.shape[0]
        nan_count = int(torch.isnan(ic).sum().item())
        # NaN 占比门槛（更直白写法）
        if nan_count >= 0.01 * T:
            return (float("-inf"), float("inf"), float("inf"), float("inf"))

        # 有效样本不足直接判无效，防抖
        valid_len = int(torch.isfinite(ic).sum().item())
        if valid_len < 5:
            return (float("-inf"), float("inf"), float("inf"), float("inf"))

        avg_ic = float(torch.nanmean(ic).item())
        if abs(avg_ic) < 1e-3:
            return (float("-inf"), float("inf"), float("inf"), float("inf"))

        valid_ic = ic[torch.isfinite(ic)]
        if valid_ic.numel() == 0:
            return (float("-inf"), float("inf"), float("inf"), float("inf"))
        std_ic = float(torch.std(valid_ic, unbiased=False).item())
        
        complexity = float(len(individual))

        return (abs(avg_ic), std_ic, float(nan_count), complexity)

    except Exception:
        # 发生真实异常（而非 0 除），返回惩罚
        return (float("-inf"), float("inf"), float("inf"), float("inf"))

# =========================
# 无效个体处理 & 多样性/表现监控
# =========================
def is_valid(ind):
    return ind.fitness.values[0] != float("-inf")

def is_valid_and_unique(new_ind, population_set):
    if not is_valid(new_ind):
        return False
    if str(new_ind) in population_set:
        return False
    return True

def ineffective_processing(population, max_attempts=100):
    """
    种群无效个体处理：持续生成有效个体来替换种群中无效个体，直到种群中所有个体都有效；
    新生成的有效个体需要不同于原始个体以保证多样性
    增加最大尝试次数，超出则跳过该位置
    """
    population_set = set(str(ind) for ind in population if is_valid(ind))
    for i, ind in enumerate(population):
        if not is_valid(ind):
            attempts = 0
            while attempts < max_attempts:
                new_ind = toolbox.individual()
                new_ind.fitness.values = toolbox.evaluate(new_ind)
                if is_valid_and_unique(new_ind, population_set):
                    population[i] = new_ind
                    population_set.add(str(new_ind))
                    break
                attempts += 1
            # 如果超出最大尝试次数还没找到有效且唯一的个体，则跳过
            if attempts >= max_attempts:
                print(f"Warning: Failed to replace invalid individual at index {i} after {max_attempts} attempts.")
    return population

def effective_ind(population):
    return len([ind for ind in population if ind.fitness.values[0] != float("-inf")]) / len(population)

def genotype_diversity(population):
    unique_individuals = set(str(ind) for ind in population if ind.fitness.values[0] != float("-inf"))
    denom = len([ind for ind in population if ind.fitness.values[0] != float("-inf")])
    return len(unique_individuals) / denom if denom > 0 else 0

def phenotype_diversity(population):
    fitness_values = [ind.fitness.values[0] for ind in population if ind.fitness.values[0] != float("-inf")]
    if len(fitness_values) <= 1:
        return 0
    return np.std(fitness_values)

def hamming_distance(ind1, ind2):
    return sum(c1 != c2 for c1, c2 in zip(str(ind1), str(ind2)))

def average_hamming_distance(population):
    total_distance = 0
    comparisons = 0
    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            if population[i].fitness.values[0] == float("-inf") or population[j].fitness.values[0] == float("-inf"):
                continue
            total_distance += hamming_distance(population[i], population[j])
            comparisons += 1
    return total_distance / comparisons if comparisons > 0 else 0

def population_avg_IC(population):
    vals = [ind.fitness.values[0] for ind in population if ind.fitness.values[0] != float("-inf")]
    return float(np.mean(vals)) if len(vals) else 0.0

def population_avg_IC_std(population):
    vals = [ind.fitness.values[1] for ind in population if ind.fitness.values[0] != float("-inf")]
    return float(np.mean(vals)) if len(vals) else 0.0

def population_avg_nan_count(population):
    vals = [ind.fitness.values[2] for ind in population if ind.fitness.values[0] != float("-inf")]
    return float(np.mean(vals)) if len(vals) else 0.0

def population_avg_complexity(population):
    vals = [ind.fitness.values[3] for ind in population if ind.fitness.values[0] != float("-inf")]
    return float(np.mean(vals)) if len(vals) else 0.0

# =========================
# 0-1 标准化 / 反归一化
# =========================
def normalizing_population(population):
    valid = [ind for ind in population if ind.fitness.values[0] != float("-inf")]
    min_f1 = min(ind.fitness.values[0] for ind in valid)
    max_f1 = max(ind.fitness.values[0] for ind in valid)
    min_f2 = min(ind.fitness.values[1] for ind in valid)
    max_f2 = max(ind.fitness.values[1] for ind in valid)
    min_f3 = min(ind.fitness.values[2] for ind in valid)
    max_f3 = max(ind.fitness.values[2] for ind in valid)
    min_f4 = min(ind.fitness.values[3] for ind in valid)
    max_f4 = max(ind.fitness.values[3] for ind in valid)

    for ind in population:
        f1 = ind.fitness.values[0]
        f2 = ind.fitness.values[1]
        f3 = ind.fitness.values[2]
        f4 = ind.fitness.values[3]
        f1_normalized = (f1 - min_f1) / (max_f1 - min_f1) if max_f1 != min_f1 else 1
        f2_normalized = (f2 - min_f2) / (max_f2 - min_f2) if max_f2 != min_f2 else 0
        f3_normalized = (f3 - min_f3) / (max_f3 - min_f3) if max_f3 != min_f3 else 0
        f4_normalized = (f4 - min_f4) / (max_f4 - min_f4) if max_f4 != min_f4 else 0
        ind.fitness.values = (f1_normalized, f2_normalized, f3_normalized, f4_normalized)

    Recover_Pac = [[min_f1, max_f1], [min_f2, max_f2], [min_f3, max_f3], [min_f4, max_f4]]
    return population, Recover_Pac

def denormalizing_population(population, Recover_Pac):
    out = []
    for ind in population:
        f1 = ind.fitness.values[0] * (Recover_Pac[0][1] - Recover_Pac[0][0]) + Recover_Pac[0][0]
        f2 = ind.fitness.values[1] * (Recover_Pac[1][1] - Recover_Pac[1][0]) + Recover_Pac[1][0]
        f3 = ind.fitness.values[2] * (Recover_Pac[2][1] - Recover_Pac[2][0]) + Recover_Pac[2][0]
        f4 = ind.fitness.values[3] * (Recover_Pac[3][1] - Recover_Pac[3][0]) + Recover_Pac[3][0]
        ind.fitness.values = (f1, f2, f3, f4)
        out.append(ind)
    return out

# =========================
# 选择函数（非支配排序 + 锦标赛）
# =========================
def tournament_selection(population, tournament_size):
    # 非支配排序
    fronts = tools.sortNondominated(population, len(population), first_front_only=False)

    # 高支配层级的个体获得更多参与锦标赛的机会
    probabilities = np.zeros(len(population))
    for i, front in enumerate(fronts):
        prob = 1.0 / (i + 1)
        for ind in front:
            probabilities[population.index(ind)] = prob
    probabilities /= probabilities.sum()  # 归一化

    selected_individuals = []
    for _ in range(POP_SIZE - NUM_ELITE):  # 种群个数 - 精英个体个数
        competitors = random.choices(population, weights=probabilities, k=tournament_size)
        best_individual = tools.selNSGA2(competitors, 1)[0]
        selected_individuals.append(best_individual)
    return selected_individuals

# =========================
# 因子重要性监控
# =========================
def factors_importance(offspring, pset):
    """
    因子重要性：自然选择出的个体中因子出现的频率（后代中包含该因子的个体个数/后代总个体数）
    """
    factors_counter = Counter()
    for individual in offspring:
        factors = []
        for node in individual:
            if isinstance(node, gp.Terminal) and not isinstance(node, (float, int)):
                if node.name[:3] == 'ARG':  # 只提取变量（因子），排除常数
                    index = int(node.name[3:])
                    renamed_name = pset.arguments[index]
                    factors.append(renamed_name)
        factors = set(factors)
        factors_counter.update(factors)
    factors_counter = dict(factors_counter)

    total_selections = len(offspring)
    factor_probabilities = {factor: count / total_selections for factor, count in factors_counter.items()}
    sorted_factors = sorted(factor_probabilities.items(), key=lambda x: x[1], reverse=True)  # 降序

    print("因子重要性:")
    for factor, prob in sorted_factors:
        print(f"{factor}: {prob:.2%}")

    return factor_probabilities

# =========================
# 点变异
# =========================
def point_mutation(individual, pset):
    """
    对个体进行点变异：选择一个随机节点，替换为新节点。
    """
    node = random.randint(0, len(individual) - 1)
    # 终端 -> 新终端
    if isinstance(individual[node], gp.Terminal):
        available_terminals = [t for t in pset.terminals[pset.ret] if isinstance(t, gp.Terminal)]
        individual[node] = random.choice(available_terminals)
    # 原始操作符 -> 同元数新操作符
    elif isinstance(individual[node], gp.Primitive):
        available_primitives = [p for p in pset.primitives[pset.ret] if p.arity == individual[node].arity]
        individual[node] = random.choice(available_primitives)
    return individual

# =========================
# 名人堂结果整理
# =========================
def df_HOF(hall_of_fame):
    ind_name = []
    IC_list = []
    IC_std_list = []
    ICIR_list = []
    nan_list = []
    complexity_list = []

    for i, ind in enumerate(hall_of_fame):
        ind_name.append(str(ind))
        IC_list.append(ind.fitness.values[0])
        IC_std_list.append(ind.fitness.values[1])
        ICIR_list.append(ind.fitness.values[0] / ind.fitness.values[1] if ind.fitness.values[1] != 0 else np.nan)
        nan_list.append(ind.fitness.values[2])
        complexity_list.append(ind.fitness.values[3])

    f_stats = pd.DataFrame({
        '个体': ind_name,
        'IC绝对值': IC_list,
        'IC_std': IC_std_list,
        'ICIR': ICIR_list,
        '空值个数': nan_list,
        '复杂度': complexity_list
    })
    return f_stats

# =========================
# 主训练流程
# =========================
def run_global_training(gpu_cache):
    """
    针对全局数据训练符号树。
    :param datasets: [(X1, y1), (X2, y2), ...]，包含所有合约的数据。
    :return: 最优个体列表、其适应度、最终种群、名人堂、名人堂因子重要性
    """
    # 初始化种群的方法
    toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # 注册遗传函数
    toolbox.register("compile", gp.compile, pset=pset)
    toolbox.register("evaluate", evaluate_individual, func=toolbox.compile, gpu_cache=gpu_cache)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("mutate", gp.mutUniform, expr=partial(gp.genHalfAndHalf, min_=1, max_=4), pset=pset)  # 子树变异
    toolbox.register("mutate_point", point_mutation, pset=pset)  # 点变异
    toolbox.register("select", tools.selTournament, tournsize=3)  # 锦标赛选择

    # 初始化种群
    population = toolbox.population(n=POP_SIZE)

    # 进化过程
    for gen in tqdm.tqdm(range(N_GENERATIONS)):
        print(f"Generation {gen + 1}:")

        t0 = time.time()
        # 评估适应度
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        torch.cuda.synchronize()
        t_eval = time.time()

        if gen == 0:
            # 初代种群无效个体处理
            print("初始种群无效个体处理中...")
            population = ineffective_processing(population)

            # 初始种群多样性检测
            effective_ind_pct = effective_ind(population)
            genotype_diversity_value = genotype_diversity(population)
            phenotype_diversity_value = phenotype_diversity(population)
            average_hamming_distance_value = average_hamming_distance(population)
            avg_ic = population_avg_IC(population)
            avg_ic_std = population_avg_IC_std(population)
            avg_nan = population_avg_nan_count(population)
            avg_complexity = population_avg_complexity(population)
            best_individual = tools.selNSGA2(population, 1)[0]

            print("种群多样性检测：")
            print(f"初始种群有效个体占比：{effective_ind_pct:.2%}")
            print(f"初始种群基因型多样性：{genotype_diversity_value:.2%}")
            print(f"初始种群表型(abs_IC_mean)多样性：{phenotype_diversity_value:.4}")
            print(f"初始种群平均汉明距离：{average_hamming_distance_value}")
            print("种群表现：")
            print(f"初始种群平均IC：{avg_ic:.2%}")
            print(f"初始种群平均IC_std：{avg_ic_std:.2%}")
            print(f"初始种群平均ICIR：{avg_ic / avg_ic_std:.4}" if avg_ic_std != 0 else "初始种群平均ICIR：NaN")
            print(f"初始种群平均空值个数：{avg_nan}")
            print(f"初始种群平均复杂度：{avg_complexity:.2f}")
            print(
                f"初始种群最优个体 : {best_individual}, "
                f"IC绝对值: {best_individual.fitness.values[0]:.2%}, "
                f"IC_std: {best_individual.fitness.values[1]:.2%}, "
                f"ICIR: {best_individual.fitness.values[0] / best_individual.fitness.values[1]:.4f} "
                if best_individual.fitness.values[1] != 0 else "ICIR: NaN"
                f", 空值个数: {best_individual.fitness.values[2]}, "
                f"复杂度: {best_individual.fitness.values[3]:.2f},"
            )

            # with open(path02 + f"{Ver}_gen{gen + 1}.pkl", 'wb') as file:
            #     pickle.dump(population, file)

        else:
            # 后续各代统计
            effective_ind_pct = effective_ind(population)
            genotype_diversity_value = genotype_diversity(population)
            phenotype_diversity_value = phenotype_diversity(population)
            average_hamming_distance_value = average_hamming_distance(population)
            avg_ic = population_avg_IC(population)
            avg_ic_std = population_avg_IC_std(population)
            avg_nan = population_avg_nan_count(population)
            avg_complexity = population_avg_complexity(population)
            best_individual = tools.selNSGA2(population, 1)[0]

            print("种群多样性检测：")
            print(f"第{gen + 1}代种群有效个体占比：{effective_ind_pct:.2%}")
            print(f"第{gen + 1}代种群基因型多样性：{genotype_diversity_value:.2%}")
            print(f"第{gen + 1}代种群表型多样性：{phenotype_diversity_value:.4}")
            print(f"第{gen + 1}代种群平均汉明距离：{average_hamming_distance_value}")
            print("种群表现：")
            print(f"第{gen + 1}代种群平均IC:{avg_ic:.2%}")
            print(f"第{gen + 1}代种群平均IC_std:{avg_ic_std:.2%}")
            print(f"第{gen + 1}代种群平均ICIR:{avg_ic / avg_ic_std:.4}" if avg_ic_std != 0 else f"第{gen + 1}代种群平均ICIR: NaN")
            print(f"第{gen + 1}代种群平均空值个数：{avg_nan}")
            print(f"第{gen + 1}代种群平均复杂度：{avg_complexity:.2f}")
            print(
                f"第{gen + 1}代种群最优个体 : {best_individual}, "
                f"IC绝对值: {best_individual.fitness.values[0]:.2%}, "
                f"IC_std: {best_individual.fitness.values[1]:.2%}, "
                f"ICIR: {best_individual.fitness.values[0] / best_individual.fitness.values[1]:.4f} "
                if best_individual.fitness.values[1] != 0 else "ICIR: NaN"
                f", 空值个数: {best_individual.fitness.values[2]}, "
                f"复杂度: {best_individual.fitness.values[3]:.2f},"
            )

            # with open(path02 + f"{Ver}_gen{gen + 1}.pkl", 'wb') as file:
            #     pickle.dump(population, file)

        # 删除无效个体
        population = [ind for ind in population if ind.fitness.values[0] != float("-inf")]

        # 标准化多目标值
        population, Recover_Pac = normalizing_population(population)

        # 使用非支配排序将种群分成多个前沿
        fronts = tools.sortNondominated(population, len(population), first_front_only=False)

        t_sort = time.time()

        print(f"有效个体可以被分为{len(fronts)}个非支配前沿")
        for rank, front in enumerate(fronts):
            print(f"第{rank + 1}层非支配层有{len(front)}个个体")

        # 更新 Hall of Fame
        best_individuals = []
        for front in fronts:
            if len(best_individuals) + len(front) > NUM_HOF:
                remaining = NUM_HOF - len(best_individuals)
                best_individuals.extend(front[:remaining])
                break
            best_individuals.extend(front)

        best_individuals = denormalizing_population(best_individuals, Recover_Pac)
        hall_of_fame.update(best_individuals)

        print("进化阶段开始...")
        # 选择父代：基于非支配排序与锦标赛选择的选择方法
        offspring = tournament_selection(population, tournament_size)

        # 克隆后代
        offspring = list(map(toolbox.clone, offspring))

        # 因子重要性检测
        importance = factors_importance(offspring, pset=pset)

        # 交叉操作
        for child1, child2 in zip(offspring[::2], offspring[1::2]):  # 后代两两配对
            if np.random.rand() < CXPB:  # CXPB 交叉概率
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # 变异操作(子树变异和点变异)
        for mutant in offspring:
            if np.random.rand() < MUT_PB:  # 子树变异概率
                toolbox.mutate(mutant)
                del mutant.fitness.values
            else:
                if np.random.rand() < MUT_Point:
                    toolbox.mutate_point(mutant)
                    del mutant.fitness.values
                else:
                    continue

        # 精英策略
        elites = best_individuals[:NUM_ELITE]
        population[:] = offspring + elites  # 更新种群

        print(f"第{gen + 1}代种群进化完毕，正在计算第{gen + 2}代种群统计值...")

        t_evo = time.time()

        print(f"[TIME] eval={t_eval - t0:.3f}s  sort={t_sort - t_eval:.3f}s  evo={t_evo - t_sort:.3f}s")

    # =========================
    # 最终代评估与输出
    # =========================
    print(f"第{N_GENERATIONS + 1}代（最终代）适应度计算中...")

    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # 删除无效个体
    population = [ind for ind in population if ind.fitness.values[0] != float("-inf")]

    # 标准化多目标值
    population, Recover_Pac = normalizing_population(population)

    # 使用非支配排序将种群分成多个前沿
    fronts = tools.sortNondominated(population, len(population), first_front_only=False)
    print(f"有效个体可以被分为{len(fronts)}个非支配前沿")
    for rank, front in enumerate(fronts):
        print(f"第{rank + 1}层非支配层有{len(front)}个个体")

    # 更新 Hall of Fame
    best_individuals = []
    for front in fronts:
        if len(best_individuals) + len(front) > NUM_HOF:
            remaining = NUM_HOF - len(best_individuals)
            best_individuals.extend(front[:remaining])
            break
        best_individuals.extend(front)

    best_individuals = denormalizing_population(best_individuals, Recover_Pac)
    hall_of_fame.update(best_individuals)

    best_individuals = hall_of_fame[:NUM_HOF]
    # 名人堂因子出现频率检测
    hof_importance = factors_importance(best_individuals, pset=pset)

    # 打印 Top 结果
    if len(best_individuals) > 0:
        top = best_individuals[0]
        denom = top.fitness.values[1]
        icir = (top.fitness.values[0] / denom) if denom != 0 else np.nan

    for i, ind in enumerate(best_individuals):
        print(
            f"Top {i + 1} 个体 : {ind}, "
            f"IC绝对值: {ind.fitness.values[0]}, "
            f"IC_std: {ind.fitness.values[1]}, "
            f"ICIR: {ind.fitness.values[0] / ind.fitness.values[1] if ind.fitness.values[1] != 0 else np.nan}, "
            f"空值个数: {ind.fitness.values[2]}, "
            f"复杂度: {ind.fitness.values[3]},"
        )

    return best_individuals, [ind.fitness.values for ind in best_individuals], population, hall_of_fame, hof_importance

共计19个因子
保留15个优秀因子：['alpha052', 'alpha013', 'alpha002', 'alpha191', 'alpha135', 'alpha022', 'alpha096', 'alpha162', 'alpha038', 'alpha164', 'alpha049', 'alpha146', 'alpha066', 'alpha161', 'alpha183']


In [4]:
import os
from typing import List, Tuple, Optional

BASE_DEFAULT = ['amount','close','high','low','open_interest','open','volume']
META_COLS = {'trade_time','datetime','contract','level_1','next_time','Y','Return'}

def _read_wide_base_csv(path: str):
    """读取类似 15m_close.csv 的“宽表”，要求有 trade_time 列，其余列是合约名。"""
    df = pd.read_csv(path)
    # 兼容 datetime 命名
    if 'trade_time' not in df.columns:
        for c in df.columns:
            if c.lower() in ('datetime','time','timestamp','dt'):
                df = df.rename(columns={c:'trade_time'})
                break
    if 'trade_time' not in df.columns:
        raise ValueError(f"{path} 缺少 trade_time/datetime 列")
    df['trade_time'] = pd.to_datetime(df['trade_time'])
    # 合约列：除 trade_time 之外的全部
    contract_cols = [c for c in df.columns if c != 'trade_time']
    return df, contract_cols

def _melt_one_feature(path: str, feat_name: str):
    """将单个基础特征宽表熔融成长表：['trade_time','contract',feat_name]"""
    wide, contracts = _read_wide_base_csv(path)
    long = wide.melt(id_vars=['trade_time'], var_name='contract', value_name=feat_name)
    long['contract'] = long['contract'].astype(str)
    return long, contracts

def _safe_cols(req: List[str], avail: List[str]) -> List[str]:
    """按原顺序取交集"""
    s = [c for c in req if c in avail]
    # 去重但保持顺序
    seen, out = set(), []
    for c in s:
        if c not in seen:
            seen.add(c); out.append(c)
    return out


def prepare_gp_datasets_from_raw_base_torch(
    factors_csv: str = 'd:/JLProject/factors/alpha191/alphas191_data.csv',
    base_dir: str = 'd:/JLProject/data_raw',
    freq_prefix: str = '15m_',
    base_cols: Optional[List[str]] = None,
    factor_list: Optional[List[str]] = None,
    time_ranges: Optional[List[Tuple[str,str]]] = None,
    contracts: Optional[List[str]] = None,
    returns_csv: Optional[str] = None,
    align: str = 'inner',
    lag_features: int = 0,
    min_rows_per_contract: int = 10,
    scale_per_contract: bool = False,
    device: Optional[str] = None,
    use_common_index: bool = True           # True: 所有合约取共同时间交集，便于做截面IC
):
    """
    返回 (gpu_cache, labeled_df, meta)
    gpu_cache 的结构：
      {
        '_names': [contract1, contract2, ...],
        '_T': 公共时间长度 T,
        '_index': DatetimeIndex(公共时间),
        '_order': X_cols(特征顺序，与 pset 一一对应),
        'IC00': ([x1_t, x2_t, ...], y_t),   # 每个是 torch.float32, device=CUDA/CPU
        'IF00': (...),
        ...
      }
    """
    # ===== 1) 读因子宽表 =====
    fac = pd.read_csv(factors_csv)
    if 'trade_time' not in fac.columns:
        raise ValueError(f"{factors_csv} 缺少 trade_time 列")
    fac['trade_time'] = pd.to_datetime(fac['trade_time'])

    contract_col = 'level_1' if 'level_1' in fac.columns else ('contract' if 'contract' in fac.columns else None)
    if contract_col is None:
        for cand in ('symbol','CODE'):
            if cand in fac.columns:
                contract_col = cand
                break
    if contract_col is None:
        raise ValueError(f"{factors_csv} 未找到合约列（默认 level_1/contract）")

    fac = fac.rename(columns={contract_col:'contract'})
    fac['contract'] = fac['contract'].astype(str)

    # ===== 2) 决定 factor_list =====
    if factor_list is None:
        factor_list = [c for c in fac.columns if c.lower().startswith('alpha')]
        factor_list.sort()
    else:
        factor_list = _safe_cols(factor_list, list(fac.columns))
        if len(factor_list) == 0:
            raise ValueError("传入的 factor_list 在因子表中一个都找不到，请检查列名。")

    # ===== 3) 读基础列并合并 =====
    if base_cols is None:
        base_cols = BASE_DEFAULT.copy()

    base_long = None
    inferred_contracts = None
    for feat in base_cols:
        path = os.path.join(base_dir, f"{freq_prefix}{feat}.csv")
        if not os.path.exists(path):
            raise FileNotFoundError(f"未找到基础数据文件：{path}")
        long_i, contracts_i = _melt_one_feature(path, feat)
        if inferred_contracts is None:
            inferred_contracts = contracts_i
        base_long = long_i if base_long is None else base_long.merge(long_i, on=['trade_time','contract'], how=align)

    # ===== 4) 合约集合 =====
    if contracts is None:
        contracts = inferred_contracts
    contracts = [str(c) for c in contracts]

    # ===== 5) (trade_time, contract) 对齐 =====
    df = fac.merge(base_long, on=['trade_time','contract'], how=align)

    # ===== 6) 时间窗 / 合约 过滤 =====
    if time_ranges:
        mask = False
        for s,e in time_ranges:
            s = pd.to_datetime(s); e = pd.to_datetime(e)
            mask = mask | ((df['trade_time']>=s) & (df['trade_time']<=e))
        df = df[mask]
    df = df[df['contract'].isin(contracts)]

    # ===== 7) 构造 Y（下一期收益）=====
    if returns_csv is not None and os.path.exists(returns_csv):
        ret = pd.read_csv(returns_csv)
        if 'trade_time' not in ret.columns:
            raise ValueError(f"{returns_csv} 缺少 trade_time 列")
        ret['trade_time'] = pd.to_datetime(ret['trade_time'])
        ret = ret.set_index('trade_time').sort_index()
        idx = ret.index
        next_map = {idx[i]: (idx[i+1] if i+1 < len(idx) else pd.NaT) for i in range(len(idx))}
        df['next_time'] = df['trade_time'].map(next_map)

        def pick_next_ret(row):
            t = row['next_time']; c = row['contract']
            if pd.isna(t) or c not in ret.columns:
                return float('nan')
            try:
                return ret.at[t, c]
            except KeyError:
                return float('nan')
        df['Y'] = df.apply(pick_next_ret, axis=1)
    else:
        close_wide, _ = _read_wide_base_csv(os.path.join(base_dir, f"{freq_prefix}close.csv"))
        open_wide,  _ = _read_wide_base_csv(os.path.join(base_dir, f"{freq_prefix}open.csv"))
        all_cons = [c for c in close_wide.columns if c != 'trade_time']
        ret_wide = close_wide.copy()
        for c in all_cons:
            if c in open_wide.columns:
                ret_wide[c] = (close_wide[c] - open_wide[c]) / open_wide[c]
            else:
                ret_wide[c] = pd.NA
        ret_wide['trade_time'] = pd.to_datetime(ret_wide['trade_time'])
        ret_wide = ret_wide.set_index('trade_time').sort_index()

        idx = ret_wide.index
        next_map = {idx[i]: (idx[i+1] if i+1 < len(idx) else pd.NaT) for i in range(len(idx))}
        df['next_time'] = df['trade_time'].map(next_map)

        def pick_next_ret2(row):
            t = row['next_time']; c = row['contract']
            if pd.isna(t) or c not in ret_wide.columns:
                return float('nan')
            try:
                return ret_wide.at[t, c]
            except KeyError:
                return float('nan')
        df['Y'] = df.apply(pick_next_ret2, axis=1)

    # ===== 8) 选列 / 去缺失 / 标准化（可选）=====
    X_cols = _safe_cols(factor_list + base_cols, list(df.columns))
    if len(X_cols) == 0:
        raise ValueError("没有可用的特征列（因子与基础列都不在数据中）。")

    if lag_features > 0:
        df = df.sort_values(['contract','trade_time'])
        df[X_cols] = df.groupby('contract', group_keys=False)[X_cols].shift(lag_features)

    df = df.dropna(subset=['trade_time','contract','Y'] + X_cols)

    if scale_per_contract:
        def _z(g):
            mu = g.mean(); sd = g.std(ddof=0).replace(0,1.0)
            return (g - mu) / sd
        df[X_cols] = df.groupby('contract', group_keys=False)[X_cols].apply(_z)

    # ===== 9) 组装 GPU 张量缓存 =====
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')

    # 仅保留样本数足够的合约
    counts = df.groupby('contract').size().to_dict()
    contracts_used = [c for c in contracts if counts.get(c, 0) >= min_rows_per_contract]
    if not contracts_used:
        raise ValueError("没有合格的合约样本，请检查时间窗、合约名以及基础/因子列。")

    # 公共时间索引（截面IC更稳）
    if use_common_index:
        common_index = None
        for c in contracts_used:
            idx_c = pd.DatetimeIndex(df.loc[df['contract'] == c, 'trade_time'].unique())
            common_index = idx_c if common_index is None else common_index.intersection(idx_c)
            # 上一行是布尔运算不合适，改为真正交集：
        if common_index is None or len(common_index) == 0:
            raise ValueError("共同时间索引为空，建议放宽 align/时间窗或改用 use_common_index=False。")

        common_index = pd.DatetimeIndex(sorted(common_index.unique()))
    else:
        common_index = pd.DatetimeIndex(sorted(df['trade_time'].unique()))
    
    df_clean = (df[['trade_time', 'contract', 'Y'] + X_cols]
                .copy()
                .astype({'contract': str}))

    # 统一到共同索引
    # 对每个列名（Y 或每个因子）各自 pivot 一次，得到 [T, N]
    gpu_cache = {}
    T = len(common_index)
    for c in contracts_used:
        g = (df[df['contract']==c]
             .set_index('trade_time')
             .reindex(common_index)    # 确保顺序完全一致
             .sort_index())
        # 若 use_common_index=True，这里一般没有缺失；否则允许 NaN 并在评估时做掩码
        Xt = [torch.tensor(g[col].to_numpy(dtype=np.float32), device=device) for col in X_cols]
        Yt = torch.tensor(g['Y'].to_numpy(dtype=np.float32), device=device)
        gpu_cache[str(c)] = (Xt, Yt)

    meta = {
        'X_cols': X_cols,
        'contracts_used': contracts_used,
        'counts': {c: int((df['contract']==c).sum()) for c in contracts_used},
        'start': common_index.min(),
        'end': common_index.max(),
        'n_rows': int(len(df)),
        'T_common': int(T),
        'device': device,
    }
    # 统一信息
    gpu_cache['_names'] = contracts_used
    gpu_cache['_T'] = T
    gpu_cache['_index'] = common_index
    gpu_cache['_order'] = X_cols

    # 也返回 labeled_df 方便自查（就是 df）
    return gpu_cache, df, meta


train_time_ranges = [
    ('2020-01-22 08:00:00','2022-01-22 15:00:00'),
    ('2023-01-01 08:00:00','2023-12-31 15:00:00'),
]

gpu_cache, labeled_df, meta = prepare_gp_datasets_from_raw_base_torch(
    factors_csv='d:/JLProject/factors/alpha191/alphas191_data.csv',
    base_dir='d:/JLProject/data_raw',
    freq_prefix='15m_',
    base_cols=base_data,                         # 你的基础列清单
    factor_list=factors_selected,                # 记得包含基础列名，和 pset 输入一一对应
    time_ranges=train_time_ranges,
    contracts=['IF00','IF01','IF02','IH00','IH01','IH02','IC00','IC01','IC02','IM00','IM01','IM02'],
    returns_csv='d:/JLProject/data_returns/15m_Returns.csv',
    align='inner',
    lag_features=0,
    scale_per_contract=False,
    device='cuda',                               # or 'cpu'
    use_common_index=True
)


In [21]:
import time
start = time.time()  # 记录开始时间
best_individuals, best_individuals_performace, population, hall_of_fame, hof_importance=run_global_training(gpu_cache)
end = time.time()    # 记录结束时间
print(f"运行时间: {(end - start):.4f} s")

  0%|          | 0/6 [00:00<?, ?it/s]

Generation 1:
初始种群无效个体处理中...
种群多样性检测：
初始种群有效个体占比：100.00%
初始种群基因型多样性：98.30%
初始种群表型(abs_IC_mean)多样性：0.007331
初始种群平均汉明距离：35.40449324662331
种群表现：
初始种群平均IC：0.93%
初始种群平均IC_std：47.99%
初始种群平均ICIR：0.01939
初始种群平均空值个数：3.994
初始种群平均复杂度：7.71
初始种群最优个体 : gp_decay_20(volume), IC绝对值: 0.73%, IC_std: 22.85%, ICIR: 0.0320 
有效个体可以被分为22个非支配前沿
第1层非支配层有68个个体
第2层非支配层有106个个体
第3层非支配层有92个个体
第4层非支配层有101个个体
第5层非支配层有124个个体
第6层非支配层有125个个体
第7层非支配层有161个个体
第8层非支配层有148个个体
第9层非支配层有151个个体
第10层非支配层有171个个体
第11层非支配层有176个个体
第12层非支配层有161个个体
第13层非支配层有123个个体
第14层非支配层有80个个体
第15层非支配层有74个个体
第16层非支配层有55个个体
第17层非支配层有28个个体
第18层非支配层有24个个体
第19层非支配层有19个个体
第20层非支配层有8个个体
第21层非支配层有4个个体
第22层非支配层有1个个体
进化阶段开始...


 17%|█▋        | 1/6 [51:07<4:15:35, 3067.18s/it]

因子重要性:
amount: 21.61%
volume: 20.83%
alpha191: 20.50%
alpha002: 19.50%
alpha164: 19.33%
alpha135: 17.39%
alpha146: 17.17%
open_interest: 17.06%
alpha013: 16.06%
alpha066: 15.11%
alpha183: 15.06%
alpha161: 14.67%
open: 14.61%
alpha038: 14.50%
alpha162: 14.44%
alpha022: 14.28%
alpha049: 13.50%
alpha052: 13.22%
alpha096: 13.17%
第1代种群进化完毕，正在计算第2代种群统计值...
[TIME] eval=1095.219s  sort=1970.425s  evo=1.540s
Generation 2:
种群多样性检测：
第2代种群有效个体占比：90.15%
第2代种群基因型多样性：84.08%
第2代种群表型多样性：0.008497
第2代种群平均汉明距离：33.239143294903116
种群表现：
第2代种群平均IC:1.07%
第2代种群平均IC_std:43.60%
第2代种群平均ICIR:0.02455
第2代种群平均空值个数：3.2540210759844705
第2代种群平均复杂度：7.75
第2代种群最优个体 : gp_decay_20(volume), IC绝对值: 0.73%, IC_std: 22.85%, ICIR: 0.0320 
有效个体可以被分为21个非支配前沿
第1层非支配层有87个个体
第2层非支配层有141个个体
第3层非支配层有172个个体
第4层非支配层有185个个体
第5层非支配层有163个个体
第6层非支配层有136个个体
第7层非支配层有142个个体
第8层非支配层有111个个体
第9层非支配层有113个个体
第10层非支配层有121个个体
第11层非支配层有120个个体
第12层非支配层有87个个体
第13层非支配层有59个个体
第14层非支配层有44个个体
第15层非支配层有21个个体
第16层非支配层有32个个体
第17层非支配层有23个个体
第18层非支配层有21个个体
第19层非支配层有

 33%|███▎      | 2/6 [1:18:28<2:28:34, 2228.64s/it]

...

 83%|████████▎ | 5/6 [3:07:40<37:28, 2248.73s/it]  

因子重要性:
volume: 39.00%
open_interest: 28.94%
alpha013: 24.89%
alpha191: 23.56%
alpha002: 23.44%
alpha164: 20.89%
amount: 19.78%
alpha135: 18.33%
open: 17.44%
alpha052: 15.50%
alpha096: 14.78%
alpha022: 14.33%
alpha066: 13.56%
alpha161: 13.39%
alpha162: 12.28%
alpha146: 12.11%
alpha183: 10.33%
alpha038: 10.17%
alpha049: 9.44%
第5代种群进化完毕，正在计算第6代种群统计值...
[TIME] eval=2451.721s  sort=23.123s  evo=0.979s
Generation 6:
种群多样性检测：
第6代种群有效个体占比：89.15%
第6代种群基因型多样性：79.58%
第6代种群表型多样性：0.01014
第6代种群平均汉明距离：37.14661099686338
种群表现：
第6代种群平均IC:1.27%
第6代种群平均IC_std:38.98%
第6代种群平均ICIR:0.03259
第6代种群平均空值个数：2.593942793045429
第6代种群平均复杂度：8.21
第6代种群最优个体 : gp_decay_20(volume), IC绝对值: 0.73%, IC_std: 22.85%, ICIR: 0.0320 
有效个体可以被分为18个非支配前沿
第1层非支配层有223个个体
第2层非支配层有289个个体
第3层非支配层有242个个体
第4层非支配层有196个个体
第5层非支配层有141个个体
第6层非支配层有122个个体
第7层非支配层有102个个体
第8层非支配层有101个个体
第9层非支配层有81个个体
第10层非支配层有59个个体
第11层非支配层有54个个体
第12层非支配层有46个个体
第13层非支配层有36个个体
第14层非支配层有21个个体
第15层非支配层有30个个体
第16层非支配层有19个个体
第17层非支配层有15个个体
第18层非支配层有6个个体
进化阶段开始...


100%|██████████| 6/6 [3:51:27<00:00, 2314.62s/it]

因子重要性:
volume: 39.83%
open_interest: 30.33%
alpha191: 26.89%
alpha013: 26.44%
alpha002: 25.22%
alpha135: 18.89%
alpha164: 18.78%
open: 18.39%
alpha052: 17.94%
alpha096: 17.22%
amount: 17.11%
alpha066: 16.06%
alpha161: 13.89%
alpha022: 12.72%
alpha038: 12.33%
alpha146: 12.00%
alpha162: 11.22%
alpha183: 10.22%
alpha049: 9.67%
第6代种群进化完毕，正在计算第7代种群统计值...
[TIME] eval=2604.081s  sort=21.911s  evo=0.929s
第7代（最终代）适应度计算中...


有效个体可以被分为16个非支配前沿
第1层非支配层有239个个体
第2层非支配层有287个个体
第3层非支配层有280个个体
第4层非支配层有222个个体
第5层非支配层有141个个体
第6层非支配层有112个个体
第7层非支配层有98个个体
第8层非支配层有82个个体
第9层非支配层有89个个体
第10层非支配层有59个个体
第11层非支配层有54个个体
第12层非支配层有46个个体
第13层非支配层有57个个体
第14层非支配层有25个个体
第15层非支配层有11个个体
第16层非支配层有3个个体
因子重要性:
alpha013: 39.00%
alpha191: 30.50%
alpha002: 24.50%
open_interest: 23.00%
alpha135: 20.50%
alpha164: 19.00%
volume: 17.50%
open: 17.50%
alpha052: 17.00%
amount: 15.00%
alpha161: 13.00%
alpha162: 12.50%
alpha038: 12.50%
alpha096: 11.00%
alpha066: 11.00%
alpha183: 9.50%
alpha049: 9.00%
alpha146: 7.50%
alpha022: 5.50%
Top 1 个体 : gp_mean2(alpha135, alpha191), IC绝对值: 0.05478474870324135, IC_std: 0.6042370200157166, ICIR: 0.0906676467817486, 空值个数: 0.0, 复杂度: 3.0,
Top 2 个体 : gp_mean2(alpha191, alpha135), IC绝对值: 0.05478474870324135, IC_std: 0.6042370200157166, ICIR: 0.0906676467817486, 空值个数: 0.0, 复杂度: 3.0,
Top 3 个体 : gp_add(alpha135, gp_tan(gp_mean2(alpha191, gp_lt(amount, volume)))), IC绝对值: 0.05303008854389191, IC_std: 0.6236928105354309,

In [12]:
print("torch version:", torch.__version__)
print("built with cuda:", torch.version.cuda)    # None 表示该 PyTorch 是 CPU-only 版本
print("is_available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("gpu count:", torch.cuda.device_count())
    print("current device:", torch.cuda.current_device())
    print("device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

torch version: 2.8.0+cu126
built with cuda: 12.6
is_available: True
gpu count: 1
current device: 0
device name: NVIDIA GeForce RTX 4060 Laptop GPU


In [13]:
c0 = gpu_cache['_names'][0]
Xt, Yt = gpu_cache[c0]
print("gpu_cache device sample:", Xt[0].device, Yt.device)  # 预期都是 cuda:0
print("order ok?:", gpu_cache['_order'] == meta['X_cols'])

gpu_cache device sample: cuda:0 cuda:0
order ok?: True


In [7]:
import torch, time

T, N = 200000, 128   # 矩阵规模大一点，能看出差别
repeat = 200        # 循环次数

def benchmark(device):
    print(f"\nRunning on {device}...")
    A = torch.randn(T, N, device=device)
    B = torch.randn(T, N, device=device)

    # 保证 GPU 上操作不会被异步延迟
    if device == "cuda":
        torch.cuda.synchronize()
    t0 = time.time()

    with torch.no_grad():
        for _ in range(repeat):
            P = A * B
            P = torch.sin(P) + torch.cos(P)
            M = torch.isfinite(P)
            P = torch.where(M, P, torch.tensor(0.0, device=device, dtype=P.dtype))
            s = (P * P).sum(dim=1)
            _ = torch.sqrt(s + 1e-8)

    if device == "cuda":
        torch.cuda.synchronize()
    print("elapsed:", time.time() - t0, "seconds")

# 先跑 CPU 版
benchmark("cpu")

# 如果有 GPU，再跑 GPU 版
if torch.cuda.is_available():
    benchmark("cuda")


Running on cpu...
elapsed: 12.635714054107666 seconds

Running on cuda...
elapsed: 2.2730209827423096 seconds
